<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/07.%20Save%20and%20load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyyaml h5py # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf

In [3]:
# to demonstrate how to save and load weights, we'll use the MNIST dataset.
# To speed up these runs, use the first 1000 examples.

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define a model

# define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# Save checkpoints during training

# You can use a trained model without having to retrain it,
# or pick-up training where you left off in case the training process was interrupted.

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

# Train the model with new callback
model.fit(train_images,
          train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback]) # Pass callback to training

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2642 - sparse_categorical_accuracy: 0.6373
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 21ms/step - loss: 1.2033 - sparse_categorical_accuracy: 0.6520 - val_loss: 0.7192 - val_sparse_categorical_accuracy: 0.7840
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4370 - sparse_categorical_accuracy: 0.8750
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4370 - sparse_categorical_accuracy: 0.8750 - val_loss: 0.5445 - val_sparse_categorical_accuracy: 0.8350
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2991 - sparse_categorical_accuracy: 0.9167
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 0.3048 - sparse_categorical_accuracy: 0.9150 - val_loss: 0.5222 - val_sparse_categorical_accuracy: 0.8300
Epoch 4/10
29/32 [=======

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Untrained model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3425 - sparse_categorical_accuracy: 0.0920 - 264ms/epoch - 8ms/step
Untrained model accuracy:  9.20%


In [8]:
# Load the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4164 - sparse_categorical_accuracy: 0.8670 - 95ms/epoch - 3ms/step
Restored model, accuracy: 86.70%


In [9]:
# Checkpoint callback options
# Train a new model, and save uniquely named checkpoints once every five epochs

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)

# create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5*n_batches
)

# create a new model instance
model = create_model()

# save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch = 0))

# train the model with new callback
model.fit(train_images,
          train_labels,
          epochs = 50,
          batch_size = batch_size,
          callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose = 0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
# Now, review the resulting checkpoints and choose the latest one
os.listdir(checkpoint_dir)

['cp-0020.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'checkpoint']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# test, reset the model, and load the latest checkpoint

# create a new model instance
model = create_model()

# load the previously saved weights
model.load_weights(latest)

# re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4717 - sparse_categorical_accuracy: 0.8780 - 188ms/epoch - 6ms/step
Restored model, accuracy: 87.80%


In [13]:
# manually save weights

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# create a new model instance
model = create_model()

# restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4717 - sparse_categorical_accuracy: 0.8780 - 225ms/epoch - 7ms/step
Restored model, accuracy: 87.80%


In [14]:
# Save the entire model

# new high level .keras format

# create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# save the entire model as a '.keras' zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1457 - sparse_categorical_accuracy: 0.6840
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4139 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2780 - sparse_categorical_accuracy: 0.9320
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2176 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1461 - sparse_categorical_accuracy: 0.9640


In [15]:
new_model = tf.keras.models.load_model('my_model.keras')

# show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# evaluate the stored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4145 - sparse_categorical_accuracy: 0.8690 - 204ms/epoch - 6ms/step
Restored model, accuracy: 86.90%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


In [17]:
# SavedModel format

# create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# save the entire model as SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.1432 - sparse_categorical_accuracy: 0.6770
Epoch 2/5
32/32 [==============================] - 0s 11ms/step - loss: 0.4337 - sparse_categorical_accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2948 - sparse_categorical_accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2113 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1599 - sparse_categorical_accuracy: 0.9630


In [19]:
# my_model directory
# ls saved_model

# contains an assets folder, save_model.pb, and variables folder.
# s saved_model/my_model

In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Restored model, accuracy: (:5.2f)%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4237 - sparse_categorical_accuracy: 0.8580 - 192ms/epoch - 6ms/step
Restored model, accuracy: (:5.2f)%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [27]:
# HDF5 format

# create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

# save the entire model to a HDF5 file.
# the '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.1799 - sparse_categorical_accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4319 - sparse_categorical_accuracy: 0.8710
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.3027 - sparse_categorical_accuracy: 0.9160
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2164 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1541 - sparse_categorical_accuracy: 0.9650


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# recreate the same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# check its accuracy
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4237 - sparse_categorical_accuracy: 0.8670 - 495ms/epoch - 15ms/step
Restored model, accuracy: 86.70%
